In [1]:
import numpy as np
import pandas as pd
from river import evaluate
from sail.models.river.forest import AdaptiveRandomForestRegressor
from sail.pipeline import SAILPipeline
from sail.utils.ts_utils import generate_features_and_targets
from sail.models.keras import OSELM
from sail.models.keras import WGLSTM
from tensorflow import keras
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sail.transformers.river.preprocessing import StandardScaler

#### Load Data


In [2]:
X = pd.read_csv("../datasets/HDWF2.csv").head(2000)

y = X["power"]
X.drop(["power", "time"], axis=1, inplace=True)

#### Model Definition


In [3]:
model = OSELM(
    loss="mae",
    optimizer=keras.optimizers.Adam,
    metrics=["accuracy"],
    epochs=1,
    verbose=0,
    num_hidden_nodes=25,
    hidden_layer_activation=tf.nn.sigmoid,
    num_output_nodes=1,
    forgetting_factor=0.5,
)

#### Create SAIL Pipeline


In [4]:
steps = [
    ("Imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("standard_scalar", StandardScaler()),
    ("regressor", model),
]
sail_pipeline = SAILPipeline(steps=steps, scoring="R2", verbosity_level=1, verbosity_interval=2)

#### Train Test Split


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#### Start Incremental Training


In [6]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, 1000, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]
    
    sail_pipeline.partial_fit(X, y)


>>> Epoch: 1 | Samples Seen: 0 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit [regressor]:  67%======     [Steps: 2/3, ETA: 00:00<00:00, Elapsed:0.010s, Batch Size=100]      

Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: mangled names are not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2023-09-26 11:50:40.647087: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.277s, Batch Size=100, P_Score=0]            


>>> Epoch: 2 | Samples Seen: 100 -------------------------------------------------------------------------------------


...

SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.048s, Batch Size=100, P_Score=0.048]            



>>> Epoch: 8 | Samples Seen: 700 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.045s, Batch Size=100, P_Score=0.0518]            



>>> Epoch: 10 | Samples Seen: 900 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.045s, Batch Size=100, P_Score=0.0656]            


#### Save SAIL Pipeline


In [7]:
sail_pipeline.save(".")

INFO:tensorflow:Assets written to: ./sail_pipeline/steps/regressor/assets


INFO:tensorflow:Assets written to: ./sail_pipeline/steps/regressor/assets
[2023-09-26 11:50:42:80] - INFO - SAIL (TF_Keras) - Model saved successfully.


'./sail_pipeline'

#### Load SAIL Pipeline


In [8]:
sail_new_pipeline = SAILPipeline.load(".")

[2023-09-26 11:50:42:237] - INFO - SAIL (TF_Keras) - Model loaded successfully.
SAIL Pipeline Progressive Score: 100%========== [Points: 900/900, Elapsed:0.0036s, Metric=R2, Batch Size=900, P_Score=0.0656]


In [9]:
sail_new_pipeline

SAILPipeline(scoring='R2',
             steps=[('Imputer', SimpleImputer()),
                    ('standard_scalar', StandardScaler()),
                    ('regressor',
                     OSELM(batch_size=None, build_fn=None, callbacks=None, loss='mae', metrics=['accuracy'], model=<keras.saving.legacy.saved_model.load._Model object at 0x2b40aa9b0>, random_state=None, run_eagerly=False, shuffle=True, validation_batch_size=None, validation_split=0.0, warm_start=False))],
             verbosity_interval=2)

In [10]:
y_preds = []
y_true = []
batch_size = 100

for start in range(1001, 1400, batch_size):

    end = start + batch_size

    X = X_train.iloc[start:end]
    y = y_train.iloc[start:end]

    sail_new_pipeline.partial_fit(X, y)


>>> Epoch: 12 | Samples Seen: 1100 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.044s, Batch Size=100, P_Score=0.0614]            



>>> Epoch: 14 | Samples Seen: 1300 -------------------------------------------------------------------------------------


SAIL Pipeline Partial fit: 100%========== [Steps: 3/3, ETA: 00:00<00:00, Elapsed:0.045s, Batch Size=99, P_Score=0.0552]            


#### Make Prediction on Hold out set


In [11]:
y_preds = []
y_true = []
batch_size = 100

for start in range(0, X_test.shape[0], batch_size):

    end = start + batch_size

    X = X_test.iloc[start:end]
    y = y_test.iloc[start:end]

    preds = sail_new_pipeline.predict(X)
    y_preds.extend(list(preds))
    y_true.extend(list(y))

#### Final Score


In [12]:
sail_new_pipeline.get_progressive_score

0.05516271961942343

In [13]:
import plotly.express as px

df = pd.DataFrame({"y_true": y_true, "y_preds": y_preds}).head(100)
fig = px.line(df, y=["y_true", "y_preds"], title='')
fig.show()